- eval:
    - test-time and train-time feed-forward
    - channel mask and recon
    - patch_embedding

In [1]:
from GeospatialFM.models.channel_vit import ChannelViTEncoder
from GeospatialFM.models.vision_transformer import ViTEncoder, ViTDecoder
import torch
from GeospatialFM.models.mae import CrossModalMAEViT
from GeospatialFM.loss.mae_loss import SpectralInterpolationLoss

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
optical_sample = torch.randn(2, 13, 224, 224)
radar_sample = torch.randn(2, 2, 224, 224)

In [64]:
optical_encoder = ChannelViTEncoder(in_chans=13, collpase_embed=False)
decoder = ViTDecoder(out_chans=1)
radar_encoder = ViTEncoder(in_chans=2)

In [65]:
196*3/4

147.0

In [66]:
148*4

592

In [67]:
output = optical_encoder.forward_encoder(optical_sample, 0.75, 0.75)

In [69]:
decoder.forward_decoder(output[0], output[2])

torch.Size([2, 588, 512])


RuntimeError: The size of tensor a (589) must match the size of tensor b (197) at non-singleton dimension 1

In [11]:
optical_encoder.num_patches / 13

196.0

In [22]:
mae = CrossModalMAEViT(optical_encoder, radar_encoder, decoder, decoder).train()

In [23]:
out = mae(optical_sample, radar_sample, 0.75, 11/12)

torch.Size([2, 196, 768]) torch.Size([2, 13])
torch.Size([2, 196, 768]) torch.Size([2, 3328])


In [29]:
out['optical_target'].shape[-1]

3328

In [7]:
optical_sample[:, 1]

tensor([[[-0.0758,  0.2432, -0.6127,  ..., -0.1294,  1.4540,  0.4133],
         [-0.0375,  0.4410,  0.3831,  ...,  0.9347,  0.1333,  0.5123],
         [ 1.0506,  0.4840, -1.4962,  ...,  0.4558, -1.2226, -1.9523],
         ...,
         [-0.8677,  0.2095,  1.0767,  ...,  0.1305, -0.4224, -1.1259],
         [ 0.1310,  0.6436,  0.3972,  ...,  0.8091,  0.2456, -0.2861],
         [-0.3486, -0.0320, -0.4587,  ...,  1.3762,  1.0520,  0.9561]],

        [[ 0.4941, -0.6912, -0.8781,  ...,  0.3940, -0.5473,  1.5865],
         [ 0.8113,  0.3332,  1.1681,  ...,  1.6995,  1.7562, -0.0785],
         [-0.5545, -0.0641, -0.0493,  ..., -1.2583,  1.1410, -0.8450],
         ...,
         [ 1.2646, -0.3340, -0.5218,  ...,  1.1230, -0.4622, -0.0545],
         [ 0.5374, -0.1509, -1.1492,  ..., -0.3861, -1.7037,  0.2794],
         [-1.3677,  0.2773,  1.6956,  ...,  0.3460,  1.3639,  0.2401]]])

In [9]:
out['optical_channel_mask'].shape

torch.Size([2, 3328])

In [39]:
out['optical_channel_mask'][1].reshape(256, 13)[0]

tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [171]:
loss = SpectralInterpolationLoss()
loss(**out, output_dict=True)

{'optical_channel_mse': tensor(0.8883, grad_fn=<MulBackward0>),
 'radar_channel_mse': tensor(0.8874, grad_fn=<MulBackward0>)}

In [160]:
for key in out.keys():
    print(key, out[key].shape)

optical_mask torch.Size([2, 196])
radar_mask torch.Size([2, 196])
optical_recon torch.Size([2, 196, 3840])
radar_recon torch.Size([2, 196, 3840])
optical_target torch.Size([2, 196, 3328])
radar_target torch.Size([2, 196, 512])
optical_cls_token torch.Size([2, 768])
radar_cls_token torch.Size([2, 768])
logit_scale torch.Size([])
optical_channel_mask torch.Size([2, 3328])


In [86]:
3840 / 15

256.0

In [112]:
optical_target = out['optical_target']
optical_recon = out['optical_recon']
radar_recon = out['radar_recon']
B, L, D = optical_target.shape

In [165]:
def _forward_channel_mse_one_modal(recon, target, mask):
    B, L, D = target.shape
    recon = recon[:, :, :D]
    loss = (recon - target).abs()
    loss = loss.mean(dim=1)
    if mask.sum() == 0:
        return loss.mean() # if no mask, mean loss on all channels
    loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
    return loss

In [162]:
optical_recon.shape, optical_target.shape

(torch.Size([2, 196, 3840]), torch.Size([2, 196, 3328]))

In [163]:
mask = out['optical_channel_mask']

In [166]:
_unimodal_channel_loss(optical_recon, optical_target, mask)

torch.Size([2, 3328]) tensor(0.8925, grad_fn=<DivBackward0>)


In [ ]:
optical_loss = optical_recon[:, :, :D]
radar_recon_o = radar_recon[:, :, :D]

In [ ]:
expanded_mask = mask.unsqueeze(1).unsqueeze(1).expand(B, L, 256, -1).reshape(B, L, D)


In [ ]:
recon_channels = (expanded_mask * radar_recon_o)
recon_channels.shape

torch.Size([2, 196, 3328])